<a href="https://colab.research.google.com/github/senudidinaya/Smart-Agri-Suite/blob/main/AI/Cultivator%20Intention%20Module/CultivatorIntent_G1_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
KAGGLE_USERNAME = "senudirupasinghe"
KAGGLE_KEY = "7780e1bc02634783fb08137fa45db94e"

import os, json, pathlib
pathlib.Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
with open("/root/.kaggle/kaggle.json","w") as f:
    json.dump({"username":KAGGLE_USERNAME,"key":KAGGLE_KEY}, f)
os.chmod("/root/.kaggle/kaggle.json", 0o600)

!pip -q install kaggle datasets soundfile librosa
!kaggle --version


Kaggle API 1.7.4.5


In [ ]:
!pip -q install --upgrade --no-cache-dir "huggingface_hub==0.25.2" "datasets==2.20.0" soundfile librosa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 179.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.25.2 which is incompatible.
diffusers 0.35.2 requires huggingface-hub>=0.34.0, but you have huggingface-hub 0.25.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompatible.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 0.25.2 which is incompatible.


In [ ]:
from getpass import getpass
from huggingface_hub import login

hf_token = getpass("Paste your Hugging Face token (starts with hf_): ")
login(token=hf_token)   # stores the token for this session


Paste your Hugging Face token (starts with hf_): ··········
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Download anchors
# Sri-Lankan English (accent):
!mkdir -p /content/data/sri_lanka_en
!kaggle datasets download -d chamodsr/sri-lankan-accent-voice-data-setenglish -p /content/data/sri_lanka_en --unzip

Dataset URL: https://www.kaggle.com/datasets/chamodsr/sri-lankan-accent-voice-data-setenglish
License(s): unknown
  0% 0.00/15.4M [00:00<?, ?B/s]
100% 15.4M/15.4M [00:00<00:00, 1.19GB/s]


In [ ]:
# Common Voice (Sinhala + English) via Kaggle big pack (then filter), or use HF datasets (faster):
# Hugging Face route
!kaggle datasets list -s "MELD" | head -n 30


ref                                                        title                                         size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
zaber666/meld-dataset                                      Multimodal EmotionLines Dataset(MELD)  11804662112  2021-02-26 14:05:51.470000           5990         38  0.6875           
argish/meld-preprocessed                                   MELD Preprocessed                       3527202381  2025-03-01 06:45:24.590000            502          2  0.625            
bhandariprakanda/meld-emotion-recognition                  MELD: Emotion Recognition Dataset      10885202576  2025-02-12 10:13:09.047000            317          1  0.64705884       
mathurinache/pretrained-models-emotion-recognition         Pretrained Models Emotion 

In [ ]:
# Install + download MELD with kagglehub
!pip -q install kagglehub
import kagglehub, os, glob

In [ ]:
path = kagglehub.dataset_download("zaber666/meld-dataset")  # returns local folder
print("Local path:", path)

100%|██████████| 11.0G/11.0G [08:17<00:00, 23.7MB/s]

Extracting files...


Local path: /root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1


In [ ]:
# See what actually came down
csvs = glob.glob(os.path.join(path,"**","*sent_emo*.csv"), recursive=True)
mp4s = glob.glob(os.path.join(path,"**","*.mp4"), recursive=True)
zips = glob.glob(os.path.join(path,"**","*.zip"), recursive=True)
print("CSV files:", len(csvs), " | MP4 files:", len(mp4s), " | ZIPs:", len(zips))

CSV files: 3  | MP4 files: 13848  | ZIPs: 0


In [ ]:
MELD_ROOT = path  # run the discovery + ensure_wav + build_manifest cells

In [ ]:
import glob, os
train_csv = glob.glob(os.path.join(MELD_ROOT,"**","train_sent_emo.csv"), recursive=True)
dev_csv   = glob.glob(os.path.join(MELD_ROOT,"**","dev_sent_emo.csv"),   recursive=True)
test_csv  = glob.glob(os.path.join(MELD_ROOT,"**","test_sent_emo.csv"),  recursive=True)
mp4s      = glob.glob(os.path.join(MELD_ROOT,"**","*.mp4"), recursive=True)
print("CSV:", bool(train_csv), bool(dev_csv), bool(test_csv), "| MP4:", len(mp4s))


CSV: True True True | MP4: 13848


In [ ]:
!apt -y install -qq ffmpeg
import os, glob, subprocess, pandas as pd, pathlib

MELD_ROOT = MELD_ROOT  # keep whatever you set earlier
AUDIO_OUT = "/content/audio/meld"
os.makedirs(AUDIO_OUT, exist_ok=True)

# Build a fast lookup once
mp4_index = {}
for p in glob.glob(os.path.join(MELD_ROOT, "**", "*.mp4"), recursive=True):
    mp4_index[os.path.basename(p).lower()] = p
print("MP4 files indexed:", len(mp4_index))

def mp4_name(dialogue_id:int, utterance_id:int):
    return f"dia{dialogue_id}_utt{utterance_id}.mp4"

def ensure_wav(dialogue_id:int, utterance_id:int):
    src = mp4_index.get(mp4_name(dialogue_id, utterance_id).lower())
    if not src:
        return None
    dst = os.path.join(AUDIO_OUT, f"{dialogue_id}_{utterance_id}.wav")
    if not os.path.exists(dst):
        subprocess.run(
            ["ffmpeg","-y","-i",src,"-vn","-ac","1","-ar","16000","-sample_fmt","s16",dst],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
    return dst if os.path.exists(dst) else None


ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
MP4 files indexed: 11264


In [ ]:
# Quick sanity check (1 clip)
import pandas as pd, glob, os

# locate CSVs
def find_csv(name, root):
    hits = glob.glob(os.path.join(root,"**",name), recursive=True)
    assert hits, f"{name} not found under {root}"
    return hits[0]

train_csv = find_csv("train_sent_emo.csv", MELD_ROOT)
df = pd.read_csv(train_csv)

# try the first row
d, u = int(df.iloc[0]["Dialogue_ID"]), int(df.iloc[0]["Utterance_ID"])
wav_path = ensure_wav(d, u)
print("WAV created:", wav_path)

WAV created: /content/audio/meld/0_0.wav


In [ ]:
# Build manifests (with progress bars)
!pip -q install tqdm
from tqdm import tqdm
import pandas as pd, os, glob

emap = {
    "engaged":    {"neutral","joy","happy","excited"},
    "hesitating": {"sadness","fear","surprise"},
    "rejecting":  {"anger","disgust","contempt"},
}
intent_id = {"engaged":0,"hesitating":1,"rejecting":2}
def emotion_to_intent(e):
    e = str(e).strip().lower()
    for k,v in emap.items():
        if e in v: return intent_id[k]
    return None

def build_manifest(csv_path, tag):
    df = pd.read_csv(csv_path)
    cols = {c.lower(): c for c in df.columns}
    D,U,E = cols["dialogue_id"], cols["utterance_id"], cols["emotion"]
    rows = []
    for d,u,e in tqdm(zip(df[D], df[U], df[E]), total=len(df), desc=f"Extract {tag}"):
        p = ensure_wav(int(d), int(u))
        y = emotion_to_intent(e)
        if p and y is not None:
            rows.append({"path": p, "label": y, "lang": "en", "domain": "general"})
    man = pd.DataFrame(rows)
    print(f"{tag}: kept {len(man)} / {len(df)}")
    return man

dev_csv  = find_csv("dev_sent_emo.csv", MELD_ROOT)
test_csv = find_csv("test_sent_emo.csv", MELD_ROOT)

man_train = build_manifest(train_csv, "train")
man_dev   = build_manifest(dev_csv,   "dev")
man_test  = build_manifest(test_csv,  "test")

man_train.to_csv("/content/manifest_meld_train.csv", index=False)
man_dev.to_csv("/content/manifest_meld_dev.csv", index=False)
man_test.to_csv("/content/manifest_meld_test.csv", index=False)

len(man_train), len(man_dev), len(man_test)

Extract train: 100%|██████████| 9989/9989 [15:14<00:00, 10.92it/s]


train: kept 9989 / 9989


Extract dev: 100%|██████████| 1109/1109 [00:55<00:00, 19.98it/s]


dev: kept 1109 / 1109


Extract test: 100%|██████████| 2610/2610 [01:30<00:00, 28.81it/s]

test: kept 2610 / 2610


(9989, 1109, 2610)

In [ ]:
# Dependancies for Phase-A
!pip -q install opensmile torch torchaudio transformers tqdm scikit-learn soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 27.6 MB/s eta 0:00:00


Wav2Vec2Processor is trying to load a tokenizer (needs a vocab.json), but facebook/wav2vec2-large-xlsr-53 doesn’t ship one—it’s a feature-extractor-only model. Use the feature extractor instead of the processor.

In [ ]:
# Install stable versions
!pip -q install --upgrade --no-cache-dir "transformers==4.39.3" "huggingface_hub>=0.23" accelerate

# 3) Use the feature extractor API (not AutoFeatureExtractor)
import torch, numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fe   = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
xlsr = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device).eval()

@torch.no_grad()
def xlsr_embed(y, sr=16000):
    inputs = fe(y, sampling_rate=sr, return_tensors="pt")
    hs = xlsr(inputs.input_values.to(device)).last_hidden_state
    return hs.mean(dim=1).squeeze(0).cpu().numpy().astype(np.float32)

In [ ]:
# Test on "Loading Model completion"
# Assuming your xlsr_embed func is defined
import torchaudio
if 'man_train' in globals():
    sample_audio, sr = torchaudio.load(man_train['path'].iloc[0])
    embed = xlsr_embed(sample_audio.numpy().squeeze(), sr)
    print("Success! Embed shape:", embed.shape)  # Expect (1024,)

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Success! Embed shape: (1024,)


In [ ]:
# simple feature builder: XLSR only
import numpy as np

def featurize(path):
    y = load_wav(path, 16000).astype(np.float32)
    return xlsr_embed(y, 16000)  # -> (1024,)

In [ ]:
# Helpers you need in the current runtime
import soundfile as sf, torch, torchaudio

def load_wav(path, sr=16000):
    y, s = sf.read(path, always_2d=False)
    # mono
    if isinstance(y, np.ndarray) and y.ndim > 1:
        y = y.mean(-1)
    # resample if needed
    if s != sr:
        y = torchaudio.functional.resample(
            torch.tensor(y, dtype=torch.float32).unsqueeze(0), s, sr
        ).squeeze(0).numpy()
    return y.astype(np.float32)

# XLSR-only feature (1024-d)
def featurize(path):
    y = load_wav(path, 16000)
    return xlsr_embed(y, 16000)  # uses the XLSR embed function you already defined

In [ ]:
# quick check
import pandas as pd
man_train = pd.read_csv("/content/manifest_meld_train.csv")
print("Feature shape:", featurize(man_train['path'].iloc[0]).shape)  # should be (1024,)

Feature shape: (1024,)


In [ ]:
# Build NPZs
from tqdm import tqdm
import pandas as pd, numpy as np, os

FEAT_DIR = "/content/features"; os.makedirs(FEAT_DIR, exist_ok=True)

def build_npz(manifest_csv, out_npz):
    man = pd.read_csv(manifest_csv)
    X, y = [], []
    for p, lab in tqdm(zip(man["path"], man["label"]), total=len(man), desc=f"Featurize {os.path.basename(manifest_csv)}"):
        x = featurize(p); X.append(x); y.append(int(lab))
    X = np.stack(X).astype(np.float32)
    y = np.array(y, dtype=np.int64)
    np.savez(out_npz, X=X, y=y)
    print(out_npz, X.shape, y.shape)

build_npz("/content/manifest_meld_train.csv", "/content/features/meld_train.npz")
build_npz("/content/manifest_meld_dev.csv",   "/content/features/meld_dev.npz")
build_npz("/content/manifest_meld_test.csv",  "/content/features/meld_test.npz")

Featurize manifest_meld_train.csv: 100%|██████████| 9989/9989 [09:13<00:00, 18.06it/s]


/content/features/meld_train.npz (9989, 1024) (9989,)


Featurize manifest_meld_dev.csv: 100%|██████████| 1109/1109 [01:00<00:00, 18.24it/s]


/content/features/meld_dev.npz (1109, 1024) (1109,)


Featurize manifest_meld_test.csv: 100%|██████████| 2610/2610 [02:39<00:00, 16.36it/s]


/content/features/meld_test.npz (2610, 1024) (2610,)


In [ ]:
# Train the head (1024 → 256 → 64 → 3)
import numpy as np, torch, torch.nn as nn, torch.optim as optim
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_npz(p):
    d=np.load(p); return d["X"].astype(np.float32), d["y"].astype(np.int64)

Xtr,ytr = load_npz("/content/features/meld_train.npz")
Xdv,ydv = load_npz("/content/features/meld_dev.npz")

tr = torch.utils.data.TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
dv = torch.utils.data.TensorDataset(torch.from_numpy(Xdv), torch.from_numpy(ydv))
train_loader = torch.utils.data.DataLoader(tr, batch_size=64, shuffle=True)
dev_loader   = torch.utils.data.DataLoader(dv, batch_size=128, shuffle=False)

class Head(nn.Module):
    def __init__(self, d_in=1024, d_h=256, d_h2=64, n_out=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in,d_h), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(d_h,d_h2), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(d_h2,n_out)
        )
    def forward(self,x): return self.net(x)

model = Head().to(device)
crit  = nn.CrossEntropyLoss()
opt   = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)

best_f1, wait, patience = 0.0, 0, 5
for ep in range(40):
    model.train()
    for xb,yb in train_loader:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(); loss = crit(model(xb), yb); loss.backward(); opt.step()
    # dev
    model.eval(); preds=[]; gold=[]
    with torch.no_grad():
        for xb,yb in dev_loader:
            pb = model(xb.to(device)).softmax(1).argmax(1).cpu().numpy()
            preds.extend(pb); gold.extend(yb.numpy())
    f1 = f1_score(gold, preds, average="macro")
    print(f"Epoch {ep:02d}  dev Macro-F1 = {f1:.3f}")
    if f1>best_f1: best_f1, wait = f1, 0; torch.save(model.state_dict(), "/content/features/head_best.pt")
    else: wait += 1
    if wait>=patience: print("Early stop."); break

print("Best dev Macro-F1:", round(best_f1,3))

Epoch 00  dev Macro-F1 = 0.242
Epoch 01  dev Macro-F1 = 0.242
Epoch 02  dev Macro-F1 = 0.242
Epoch 03  dev Macro-F1 = 0.242
Epoch 04  dev Macro-F1 = 0.242
Epoch 05  dev Macro-F1 = 0.242
Early stop.
Best dev Macro-F1: 0.242


In [ ]:
# Test set evaluation
from sklearn.metrics import classification_report, confusion_matrix
Xte,yte = load_npz("/content/features/meld_test.npz")
te_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(Xte), torch.from_numpy(yte)),
    batch_size=128, shuffle=False
)

model.load_state_dict(torch.load("/content/features/head_best.pt", map_location=device))
model.eval(); preds=[]; gold=[]; probs=[]
with torch.no_grad():
    for xb,yb in te_loader:
        pr = model(xb.to(device)).softmax(1).cpu().numpy()
        probs.append(pr); preds.extend(pr.argmax(1)); gold.extend(yb.numpy())
probs = np.vstack(probs)

print(classification_report(gold, preds, target_names=["Engaged","Hesitating","Rejecting"]))
print("Confusion matrix:\n", confusion_matrix(gold, preds))

              precision    recall  f1-score   support

     Engaged       0.64      1.00      0.78      1658
  Hesitating       0.00      0.00      0.00       539
   Rejecting       0.00      0.00      0.00       413

    accuracy                           0.64      2610
   macro avg       0.21      0.33      0.26      2610
weighted avg       0.40      0.64      0.49      2610

Confusion matrix:
 [[1658    0    0]
 [ 539    0    0]
 [ 413    0    0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Check class balance
import numpy as np
dtr = np.load("/content/features/meld_train.npz"); ytr = dtr["y"]
ddv = np.load("/content/features/meld_dev.npz");   ydv = ddv["y"]
dte = np.load("/content/features/meld_test.npz");  yte = dte["y"]

def counts(y):
    u,c = np.unique(y, return_counts=True);
    return dict(zip(u,c)), c.sum()

print("Train:", counts(ytr))  # {0:…,1:…,2:…}, total
print("Dev:  ", counts(ydv))
print("Test: ", counts(yte))


Train: ({np.int64(0): np.int64(6453), np.int64(1): np.int64(2156), np.int64(2): np.int64(1380)}, np.int64(9989))
Dev:   ({np.int64(0): np.int64(633), np.int64(1): np.int64(301), np.int64(2): np.int64(175)}, np.int64(1109))
Test:  ({np.int64(0): np.int64(1658), np.int64(1): np.int64(539), np.int64(2): np.int64(413)}, np.int64(2610))


In [ ]:
import numpy as np, torch, torch.nn as nn, torch.optim as optim
from sklearn.metrics import f1_score
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Xtr,ytr = np.load("/content/features/meld_train.npz")["X"], np.load("/content/features/meld_train.npz")["y"]
Xdv,ydv = np.load("/content/features/meld_dev.npz")["X"],   np.load("/content/features/meld_dev.npz")["y"]

tr = TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
dv = TensorDataset(torch.from_numpy(Xdv), torch.from_numpy(ydv))

# class weights (inverse freq)
counts = np.bincount(ytr, minlength=3).astype(np.float32)
class_w = (counts.sum() / (3.0 * np.maximum(counts, 1)))  # balances classes
cw = torch.tensor(class_w, dtype=torch.float32, device=device)

# balanced sampling
sample_w = class_w[ytr]
sampler = WeightedRandomSampler(weights=torch.from_numpy(sample_w), num_samples=len(sample_w), replacement=True)

train_loader = DataLoader(tr, batch_size=64, sampler=sampler)
dev_loader   = DataLoader(dv, batch_size=128, shuffle=False)

class Head(nn.Module):
    def __init__(self, d_in=1024, d_h=256, d_h2=64, n_out=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in,d_h), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(d_h,d_h2), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(d_h2,n_out)
        )
    def forward(self,x): return self.net(x)

model = Head().to(device)
crit  = nn.CrossEntropyLoss(weight=cw)  # <= weighted loss
opt   = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

best_f1, wait, patience = 0, 0, 6
for ep in range(60):
    model.train()
    for xb,yb in train_loader:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(); loss = crit(model(xb), yb); loss.backward(); opt.step()
    # dev macro-F1
    model.eval(); preds=[]; gold=[]
    with torch.no_grad():
        for xb,yb in dev_loader:
            pb = model(xb.to(device)).softmax(1).argmax(1).cpu().numpy()
            preds.extend(pb); gold.extend(yb.numpy())
    f1 = f1_score(gold, preds, average="macro")
    print(f"Epoch {ep:02d}  dev Macro-F1 = {f1:.3f}")
    if f1 > best_f1: best_f1, wait = f1, 0; torch.save(model.state_dict(), "/content/features/head_best.pt")
    else: wait += 1
    if wait >= patience: print("Early stop."); break

print("Best dev Macro-F1:", round(best_f1,3))
